# N-gramy słów w klasyfikacji na przykładzie SVC

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_20newsgroups # zbiór danych zawarty w Sklearn, który zawiera dane z 20 grup newsowych
import numpy as np


np.random.seed(0) # ustawienie seed na 0

categories = ['sci.space', 'comp.graphics', 'talk.politics.misc', 'comp.sys.mac.hardware'] # kategorie do analizy

train = fetch_20newsgroups(subset='train',
                                   categories=categories,
                                   shuffle=True,
                                   random_state=42) # pobranie zbioru treningowego
    

test = fetch_20newsgroups(subset='test',
                                  categories=categories,
                                  shuffle=True,
                                  random_state=42) # pobranie zbioru testowego



pipeline = Pipeline([             # stworzenie pipeline'u: surowy tekst -> TFIDF vectorizer -> klasyfikator 
    ('tfidf', TfidfVectorizer(max_df=0.1, ngram_range=(1,2))),
    ('clf', SVC(C=1.0, kernel='linear')),
])


pipeline.fit(train.data, train.target) # wektoryzacja danych i trenowanie klasyfikatora na zbiorze treningowym

print("W słowniku znajduje się {n} różnych cech".format(
    n=len(pipeline.named_steps['tfidf'].vocabulary_.keys())))


print(classification_report(test.target, pipeline.predict(test.data))) # testowanie klasyfikatora - szerokie podsumowanie uwzględniające miary: precision, recall, f1

W słowniku znajduje się 287718 różnych cech
              precision    recall  f1-score   support

           0       0.87      0.94      0.91       389
           1       0.92      0.94      0.93       385
           2       0.95      0.90      0.92       394
           3       0.97      0.90      0.93       310

    accuracy                           0.92      1478
   macro avg       0.93      0.92      0.92      1478
weighted avg       0.92      0.92      0.92      1478



# Wykorzystanie n-gramów w klasyfikacji - detekcja języka

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import pandas
import numpy as np


def get_class_name_from_id(ids, mapping): # funkcja mapująca identyfikator liczbowy kategorii na wartość tekstową, np: 0->"polish", 1->"english"
    return [mapping[id] for id in ids]


full_dataset = pandas.read_csv('language_detection_1000.csv', encoding='utf-8') # wczytanie danych z pliku CSV
lang_to_id = {'polish': 0, 'english': 1, 'french': 2,
              'german': 3, 'italian': 4, 'spanish': 5}
id_to_lang = {v: k for k,v in lang_to_id.items()}
full_dataset['label_num'] = full_dataset.lang.map(lang_to_id)  # mapowanie wartości na liczby 

np.random.seed(0)                                       # ustawienie seed na 0
train_indices = np.random.rand(len(full_dataset)) < 0.7 # wylosowanie 70% wierszy, które znajdą się w zbiorze treningowym

train = full_dataset[train_indices] # wybranie zbioru treningowego
test = full_dataset[~train_indices] # wybranie zbioru testowego


pipeline = Pipeline([             # stworzenie pipeline'u: surowy tekst -> TFIDF vectorizer -> klasyfikator 
    ('tfidf', TfidfVectorizer(max_features=300, ngram_range=(2,2), analyzer="char")),
    ('scaler', StandardScaler(with_mean = False)),
    ('clf', LogisticRegression()),
])


pipeline.fit(train['text'], train['label_num']) # wektoryzacja danych i trening klasyfikatora

print("Oto kilka przykładowych cech stworzonych przez TfidfVectorizer: {n}".format(
    n=list(pipeline.named_steps['tfidf'].vocabulary_.keys())[:5]))


text_to_predict = "Bonjour!" # sprawdzenie działania
predicted = pipeline.predict([text_to_predict])
print("\n\nTekst: {t} został zaklasyfikowany jako: {p}\n\n".format(
    t=text_to_predict,
    p=id_to_lang[predicted[0]]
))


print(classification_report( # ocena klasyfikatora
    get_class_name_from_id(test['label_num'], id_to_lang), 
    get_class_name_from_id(pipeline.predict(test['text']), id_to_lang)
))

Oto kilka przykładowych cech stworzonych przez TfidfVectorizer: ['ap', 'pe', 'el', 'lu', 'je']


Tekst: Bonjour! został zaklasyfikowany jako: french


              precision    recall  f1-score   support

     english       1.00      1.00      1.00       303
      french       1.00      1.00      1.00       280
      german       1.00      1.00      1.00       337
     italian       1.00      1.00      1.00       273
      polish       1.00      1.00      1.00       291
     spanish       1.00      1.00      1.00       299

    accuracy                           1.00      1783
   macro avg       1.00      1.00      1.00      1783
weighted avg       1.00      1.00      1.00      1783



## Sprawdzenie istotności cech

In [3]:
def language_indicators(feature_names, feature_importances, id_to_lang): # funkcja oceniająca cechy najsilniej skojarzone z klasami
    for i, language in enumerate(feature_importances): # iteracja po macierzy feature_importances (wymiarów: język x cechy) wierszami (język po języku)
        scored_features = list(zip(feature_names, language)) # tworzenie skojarzenia nazw cech z wagami modelu (ponieważ używamy regresji logistycznej - każda cecha ma swoją wagę, która jest optymalizowana w procesie uczenia. Każda klasa ma osobny model ze swoimi wagami)
        scored_features = sorted(scored_features, key=lambda x: x[1], reverse=True) # sortowanie cech skojarzonych z wagami malejąco 
        print("W rozpoznaniu języka {lang} najważniejsze cechy to:".format(
            lang=id_to_lang[i]) # zamiana identyfikatora na nazwę języka
        )
        for feature, score in scored_features[:5]: # wybór 5 najważniejszych cech
            print("\t'{feature}': {score}".format(feature=feature, score=score))
        

language_indicators( # wyświetlenie najważniejszych cech dla każdej kategorii
    pipeline.named_steps['tfidf'].get_feature_names_out(), # pobranie nazwy cechy
    pipeline.named_steps['clf'].coef_, # pobranie wyuczonych współczynnikiów 
    id_to_lang # mapowanie z identyfikatora numerycznego na pełną nazwę języka
)

W rozpoznaniu języka polish najważniejsze cechy to:
	'dz': 0.19163713727725623
	'cz': 0.1649009131198807
	'ię': 0.1640251859302473
	'ow': 0.16229669569677055
	'wa': 0.15187384680981408
W rozpoznaniu języka english najważniejsze cechy to:
	'th': 0.42383859836240256
	' t': 0.37872218470723895
	'd ': 0.28129480440165094
	'of': 0.25858398649029607
	'y ': 0.2520002950043387
W rozpoznaniu języka french najważniejsze cechy to:
	't ': 0.278915327225123
	'té': 0.26532601045539983
	'ré': 0.24798957436559332
	'ai': 0.24521470119052077
	'ce': 0.23995343713432304
W rozpoznaniu języka german najważniejsze cechy to:
	'ei': 0.25373304796517443
	'au': 0.23734904149545072
	'er': 0.23458829477837226
	'ch': 0.23439514106726453
	'en': 0.2275646227666411
W rozpoznaniu języka italian najważniejsze cechy to:
	'i ': 0.4288409677099189
	'o ': 0.31534222725009353
	'zi': 0.28110382653414995
	'tt': 0.27269593895744765
	'll': 0.27156186429069823
W rozpoznaniu języka spanish najważniejsze cechy to:
	'os': 0.30009447

# Wykorzystanie n-gramów słów w generowaniu tekstu

In [4]:
def get_word_ngrams(data, n_gram_len): # funkcja dzieląca tekst na n-gramy słów 
    splitted = data.split()
    result = []
    for i in range(len(splitted) - n_gram_len + 1):
        tmp = []
        for j in range(n_gram_len):
            tmp.append(splitted[i+j])
        result.append(tmp)
    return result
print(get_word_ngrams("The big brown fox jumped over the fence.", 3))

[['The', 'big', 'brown'], ['big', 'brown', 'fox'], ['brown', 'fox', 'jumped'], ['fox', 'jumped', 'over'], ['jumped', 'over', 'the'], ['over', 'the', 'fence.']]


## Generowanie tekstu metodą Markova korzystając z n-gramów słów

In [5]:
from collections import Counter
import random
import itertools

def generate_ngram_markov(n_gram_len):
    markov_dict = dict() # stworzenie słownika, który wskaże listę dozwolonych słów po zaobserwowanych n-poprzednich słowach.
    with open("polish_europarl.txt", 'r', encoding='utf8') as f: # wczytanie korpusu danych
        data = f.read().lower()                                  # zamiana wszystkich wielkich liter na małe
        n_grams = get_word_ngrams(data, n_gram_len)              # wygenerowanie wszystkich n-gramów słów z korpusu
        for n_gram in n_grams:
            context = " ".join(n_gram[:-1])      # połączenie wszystkich słów z n-gramu poza ostatnim w 1 string
            last_word = str(n_gram[-1])
            
            if context not in markov_dict.keys(): # jeśli n-gram bez ostatniego słowa nie występuje w słowniku,
                markov_dict[context] = list()     # należy dopisać go do słownika i stworzyć mu listę
            markov_dict[context].append(last_word) # wiedząc, że ubiedzony n-gram jest w słowniku, należy dopisać ostatnie słowo do listy
    
    for context in markov_dict.keys():
        markov_dict[context] = Counter(markov_dict[context])  # stworzenie histogramu słów jakie występują w korpusie po kontekście
    
    return markov_dict


n_gram_len = 3  # liczba słów do stworznia n-gramu
markov_dict = generate_ngram_markov(n_gram_len)  # stworzenie słownika z histogramami słów dla poszczególnych kontekstów

text = 'Średnio co dwa' # tekst, od którego rozpocznie się generowanie

for i in range(500):
    text_spl = text.split(" ")     # podzielenie istniejącego tekstu po spacji
    context = " ".join(text_spl[-n_gram_len+1:])   # pobranie ostatnich n_gram_len - 1 słów
    idx = random.randrange(sum(markov_dict[context].values())) # sprawdenie dozwolonych słów jako następniki kontekstu (context) i wybór następnika, który zostanie wylosowany zgodnie z rozkładem stworzonym przez histogram
    new_word = next(itertools.islice(markov_dict[context].elements(), idx, None)) # wybranie wylosowanego słowa
    text = text + " " + new_word # doklejenie wylosowanego słowa
print(text)

Średnio co dwa miesiące od początku roku - wskutek opóźnienia projektów inwestycyjnych, zwłaszcza nabucco i innych krajów. zasadniczo istnieje wysokie ryzyko omijania przepisów krajowych, ma ogromne znaczenie w tych negocjacjach, która cechuje się delikatnością i stanowi zagrożenie dla gospodarki. niestety, bezrobocie w młodym wieku ma długotrwałe i negatywne skutki. niestety, chociaż jest ona potrzebna. ue udowodniła to raz jeszcze, niezwłocznie wypłacając środki pomocowe w krajach rozwijających się. dla obywateli ze wschodu i z wynikami odnośnych ocen rocznych. w tym konkretnym przypadku, ale chciałbym dotknąć dwóch rzeczy. ze zbliżającym się przyjęciem konwencji mop w sprawie sudanu, koncentrując się głównie na stworzeniu środowiska, w którym zauważa, że wciąż zdarza się, iż takie właśnie stanowisko chcemy zająć. musimy jednak wykazać poczucie odpowiedzialności wśród zainteresowanych stron powinien przyczynić się do wzmożonego ataku na sektor publiczny; zawiera ono szereg środków, k

# Wykorzystanie n-gramów znaków w generowaniu tekstu

In [6]:
def get_character_ngrams(data, n_gram_len): # funkcja dzieląca tekst na n-gramy znaków
    result = []
    for i in range(len(data) - n_gram_len + 1):
        result.append(data[i:i+n_gram_len])
    return result
print(get_character_ngrams("The big brown fox jumped over the fence.", 3))

['The', 'he ', 'e b', ' bi', 'big', 'ig ', 'g b', ' br', 'bro', 'row', 'own', 'wn ', 'n f', ' fo', 'fox', 'ox ', 'x j', ' ju', 'jum', 'ump', 'mpe', 'ped', 'ed ', 'd o', ' ov', 'ove', 'ver', 'er ', 'r t', ' th', 'the', 'he ', 'e f', ' fe', 'fen', 'enc', 'nce', 'ce.']


## Generowanie tekstu metodą Markova korzystając z n-gramów znaków

In [7]:
from collections import Counter
import random
import itertools

def generate_ngram_markov(n_gram_len):
    markov_dict = dict() # stworzenie słownik, który wskaże listę dozwolonych słów po zaobserwowanych n-poprzednich słowach
    with open('pan_tadeusz.txt', 'r', encoding='utf8') as f: # wczytanie korpusu danych
        data = f.read().lower()                              # zamiana wszystkich wielkich liter na małe
        n_grams = get_character_ngrams(data, n_gram_len)     # wygenerowanie wszystkich n-gramów słów z korpusu
        for n_gram in n_grams:
            context = n_gram[:-1]
            last_char = n_gram[-1]
            if context not in markov_dict.keys(): # jeśli n-gram bez ostatniego znaku nie występuje jeszcze w słowniku,
                markov_dict[context] = list()     # należy dopisać go do słownika i stwórzyć mu listę
            markov_dict[context].append(last_char) # wiedząc, że ubiedzony n-gram jest w słowniku, należy dopisać ostatni znak do listy
    
    for context in markov_dict.keys():
        markov_dict[context] = Counter(markov_dict[context])  # stworzenie histogramu liter jakie występują w korpusie po kontekście
    
    return markov_dict


text = 'U szlachty' # tekst, od którego rozpocznie się generowanie
n_gram_len = len(text)  # liczba znaków do stworznia n-gramu
markov_dict = generate_ngram_markov(n_gram_len)  # stworzenie słownika z histogramami słów dla poszczególnych kontekstów

for i in range(500):
    context = text[-n_gram_len+1:]   # pobranie ostatnich n_gram_len - 1 słów
    idx = random.randrange(sum(markov_dict[context].values())) # sprawdzenie dozwolonych słów jako następniki kontekstu i wybór takiego, który zostanie wylosowany zgodnie z rozkładem stworzonym przez histogram
    new_char = next(itertools.islice(markov_dict[context].elements(), idx, None)) # wybranie wylosowanego słowa
    text = text + new_char # doklejenie wylosowanego znaku na końcu

print(text)

U szlachty tłuszcza;
bo pan bóg, kiedy karę na naród przypuszcza,
zmykali. zwierz zwraca się czasem,
spojrzy, klapnie paszczęki na nogach się opierał ryków — ot słowo! co po waszej zgubie?
ja człek podeszły w lata,
w podróżach swych zbawców zobaczy.
sędzia poznał: «jak się masz, mój jaśnie wielmożnej tytuł przybrać miała,
a znów tylko brzęk usłyszy i rymów porządek, lecz go tam nie rozpusta płocha,
lecz mniej wielkie, mniej pilni.

    tymczasem buchman radości, niestety, sprzątniono…
a i to, bóg mi świad


In [8]:
# Zad 5:
#  Pytanie 1:  Przy zbyt krótkich n-gramach występuje ryzyko stworzenia tekstu, którego kolejne słowa słowa lub głoski bedą
# wydawały się losowe.
#  Pytanie 2:  Przy zbyt długich n-gramach występuje ryzyko stworzenia tekstu, który będzie bardzo podobny do tekstu 
# oryginalnego.